In [7]:
import streamlit as st

import openai
import os
import pandas as pd
import ast

from dotenv import load_dotenv

import numpy as np
from numpy import dot
from numpy.linalg import norm

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [8]:
def get_embedding(text):
    response = client.embeddings.create(
        input=[text],
        model='text-embedding-ada-002'
    )
    return response.data[0].embedding

In [9]:
folder_path = './data'
file_name = 'embedding.csv'
file_path = os.path.join(folder_path, file_name)

# embedding.csv 파일을 읽어서 DataFrame으로 변환
df = pd.read_csv(file_path)
df['embedding'].head(3)
df['embedding'] = df['embedding'].apply(ast.literal_eval)
df['embedding'].head(3)

0    [-0.0034950384870171547, -0.013230260461568832...
1    [-0.0025564832612872124, -0.000404238875489681...
2    [0.008293657563626766, -0.02376808412373066, 0...
Name: embedding, dtype: object

In [10]:
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

def return_answer_candidate(df, query):
    query_embedding = get_embedding(query)
    df["similarity"] = df.embedding.apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))
    top_three_doc = df.sort_values("similarity", ascending=False).head(3)
    return top_three_doc

# 챗봇의 답변을 만들기 위해 사용될 프롬프트를 만드는 함수.
def create_prompt(df, query):
    result = return_answer_candidate(df, query)
    system_role = f"""You are an artificial intelligence language model that specializes in summarizing \
    and answering documents about Seoul's youth policy.
    You need to take a given document and return a very detailed summary of the document in the query language.
    Here are the document: 
            doc 1 :{str(result.iloc[0]['text'])}
            doc 2 :{str(result.iloc[1]['text'])}
            doc 3 :{str(result.iloc[2]['text'])}
    You must return in Korean. Return a accurate answer based on the document.
    """
    user_content = f"""User question: "{str(query)}". """

    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_content}
    ] 
    return messages


In [11]:
user_input = "신혼부부 주거 지원 서울시 정책은?"

prompt = create_prompt(df, user_input)

In [12]:
df

,text,embedding,similarity
0,서울시에서는 안심소득 참여가구를 모집합니다. 이 프로그램은 기준소득 대비 부족한 가...,"[-0.0034950384870171547, -0.013230260461568832...",0.838804
1,"기후동행카드는 서울시에서 제공하는 대중교통 정기권으로, 서울 지역 내의 지하철, 시...","[-0.0025564832612872124, -0.000404238875489681...",0.817272
2,서울시는 청년들의 학습 어려움을 해소하고 대학생들에게 지식을 나누는 기회를 제공하는...,"[0.008293657563626766, -0.02376808412373066, 0...",0.830208
3,"서울형 뉴딜 일자리는 서울시에서 운영하는 프로그램으로, 일 경험과 취업 역량을 키울...","[-0.014822343364357948, -0.027527209371328354,...",0.835217
4,"서울시 청년동행센터는 청년들을 위한 정책 유형, 주관 기관, 지원 내용, 사업운영기...","[0.0018770467722788453, -0.0012499765725806355...",0.848737
5,"기후동행카드는 서울시에서 운영하는 대중교통 정기권 프로그램으로, 서울 권역 내 모든...","[-0.001742577995173633, -0.006159506738185883,...",0.815602
6,서울시에서는 서울런 멘토단을 정기모집하고 있습니다. 이 프로그램은 청년들의 학습 어...,"[0.0004731236258521676, -0.02350677363574505, ...",0.839869
7,서울시 청년동행센터는 청년들을 위한 지원 프로그램을 운영하고 있습니다. 복지 및 문...,"[0.009518656879663467, 0.010817860253155231, 0...",0.840423
8,서울시에서는 안심소득 참여가구를 모집하고 있습니다. 이 프로그램은 기준소득 대비 부...,"[-0.0013407586375251412, -0.013174484483897686...",0.847354
9,"서울형 뉴딜 일자리는 서울시에서 진행하는 프로그램으로, 일 경험과 취업 역량을 키울...","[-0.00925271213054657, -0.025175366550683975, ...",0.844360


In [13]:
df.sort_values("similarity", ascending=False).head(3)

,text,embedding,similarity
18,서울시에서는 안심소득 참여가구를 모집합니다. 이 프로그램은 기준소득 대비 부족한 가...,"[0.005462611559778452, -0.01302958931773901, 0...",0.855075
4,"서울시 청년동행센터는 청년들을 위한 정책 유형, 주관 기관, 지원 내용, 사업운영기...","[0.0018770467722788453, -0.0012499765725806355...",0.848737
8,서울시에서는 안심소득 참여가구를 모집하고 있습니다. 이 프로그램은 기준소득 대비 부...,"[-0.0013407586375251412, -0.013174484483897686...",0.847354
